**Autoregressive model as a benchmark**

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stat
from scipy.stats import norm

# Import pytorch utilities
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [3]:
x_train = pd.read_csv('windforecasts_wf1.csv', index_col='date')
y_train = pd.read_csv('train.csv')
# just consider the wind farm 1

In [4]:
y_train['date'] = pd.to_datetime(y_train.date, format= '%Y%m%d%H')
y_train.index = y_train['date'] 
y_train.drop('date', inplace = True, axis = 1)

In [5]:
# Use only the power time series when continuous

complete_ts = y_train[:'2011-01-01 00'] # all the data without any gaps
input_generator = np.transpose(np.array([complete_ts.wp1]))
length = 24 # length of the time series, PARAMETER TO TUNE

In [6]:
# Define the validation set as one sequence
validation_power = input_generator[int(len(input_generator)*0.8)+1 : int(len(input_generator))-1]

In [22]:
# Define slices of 24h inputs and corresponding targets 1, 2 and 3 hours ahead
p_inputs = []
p_targets1h = []
p_targets2h = []
p_targets3h = []
p_targets4h = []
p_targets5h = []
p_targets6h = []
for i in range(33,len(validation_power)-8):
  p_inputs.append(validation_power[i:i+3])
  p_targets1h.append(validation_power[i+3])
  p_targets2h.append(validation_power[i+4])
  p_targets3h.append(validation_power[i+5])
  p_targets4h.append(validation_power[i+6])
  p_targets5h.append(validation_power[i+7])
  p_targets6h.append(validation_power[i+8])

In [21]:
p_targets1h[-1]

array([0.677])

In [8]:
# Definition of ARIMA model (was fitted in Matlab using the training set)
# P_t = a + b*P_{t-1} + c*P_{t-2} + d*P{t-3}
a = 0.0139
b = 1.189
c = -0.282
d = 0.0361

In [23]:
# Forecasting 1, 2 and 3 hours ahead

# Store predictions and errors
pred_1h = []
err_1h = []
pred_2h = []
err_2h = []
pred_3h = []
err_3h = []
pred_4h = []
err_4h = []
pred_5h = []
err_5h = []
pred_6h = []
err_6h = []

# Loop over the sequences of valid data
for seq in range(len(p_inputs)):

    # Define past value for the 1h forecast
    past = p_inputs[seq]
    
    # Take ARIMA output for the past sequence
    pred_1h.append(a + b*past[2] + c*past[1] + d*past[0])
    err_1h.append(pred_1h[-1][0]-p_targets1h[seq][0])

    # Repeat with prediction 2 hours ahead actualizing the past values
    past = np.append(past,[pred_1h[-1]],0)
    pred_2h.append(a + b*past[3] + c*past[2] + d*past[1])
    err_2h.append(pred_2h[-1][0]-p_targets2h[seq][0])

    # Repeat with prediction 3 hours ahead
    past = np.append(past,[pred_2h[-1]],0)
    pred_3h.append(a + b*past[4] + c*past[3] + d*past[2])
    err_3h.append(pred_3h[-1][0]-p_targets3h[seq][0])

    # Repeat with prediction 4 hours ahead
    past = np.append(past,[pred_3h[-1]],0)
    pred_4h.append(a + b*past[5] + c*past[4] + d*past[3])
    err_4h.append(pred_4h[-1][0]-p_targets4h[seq][0])

    # Repeat with prediction 5 hours ahead
    past = np.append(past,[pred_4h[-1]],0)
    pred_5h.append(a + b*past[6] + c*past[5] + d*past[4])
    err_5h.append(pred_5h[-1][0]-p_targets5h[seq][0])

    # Repeat with prediction 6 hours ahead
    past = np.append(past,[pred_5h[-1]],0)
    pred_6h.append(a + b*past[7] + c*past[6] + d*past[5])
    err_6h.append(pred_6h[-1][0]-p_targets6h[seq][0])

    if seq % 100 == 0:
      print(f'step {seq+1}, RMSE 1h: {np.sqrt(stat.mean(err_1h[n]**2 for n in range(len(err_1h))))}, RMSE 2h: {np.sqrt(stat.mean(err_2h[n]**2 for n in range(len(err_2h))))}, RMSE 3h: {np.sqrt(stat.mean(err_3h[n]**2 for n in range(len(err_3h))))}, RMSE 4h: {np.sqrt(stat.mean(err_4h[n]**2 for n in range(len(err_4h))))}, RMSE 5h: {np.sqrt(stat.mean(err_5h[n]**2 for n in range(len(err_5h))))}, RMSE 6h: {np.sqrt(stat.mean(err_6h[n]**2 for n in range(len(err_6h))))}')

step 1, RMSE 1h: 0.045467399999999936, RMSE 2h: 0.10874863859999984, RMSE 3h: 0.15444422449539985, RMSE 4h: 0.21930133997983037, RMSE 5h: 0.28430424778177554, RMSE 6h: 0.14661510924250282
step 101, RMSE 1h: 0.0628139123390316, RMSE 2h: 0.10276508405052105, RMSE 3h: 0.12504098913512904, RMSE 4h: 0.1382494841424183, RMSE 5h: 0.14921857708087624, RMSE 6h: 0.15983270281782166
step 201, RMSE 1h: 0.06971821161399318, RMSE 2h: 0.10947304418612125, RMSE 3h: 0.13429017806505814, RMSE 4h: 0.14839147579856782, RMSE 5h: 0.16122427273771917, RMSE 6h: 0.17508801371820407
step 301, RMSE 1h: 0.07147181316481051, RMSE 2h: 0.11223945350982892, RMSE 3h: 0.13766539152272722, RMSE 4h: 0.1537785637890471, RMSE 5h: 0.1669446258916058, RMSE 6h: 0.18173493526389065
step 401, RMSE 1h: 0.06861351306619791, RMSE 2h: 0.1080009904547911, RMSE 3h: 0.13330212352764972, RMSE 4h: 0.14991575000444715, RMSE 5h: 0.1635291678958624, RMSE 6h: 0.17746205607212523
step 501, RMSE 1h: 0.06617313114720814, RMSE 2h: 0.10358291756

In [24]:
# Estimation of confidence intervals:
RMSE_1h = np.sqrt(stat.mean(err_1h[n]**2 for n in range(len(err_1h))))
RMSE_2h = np.sqrt(stat.mean(err_2h[n]**2 for n in range(len(err_2h))))
RMSE_3h = np.sqrt(stat.mean(err_3h[n]**2 for n in range(len(err_3h))))
CI_1h = [norm.ppf(0.025)*RMSE_1h,norm.ppf(0.975)*RMSE_1h]
CI_2h = [norm.ppf(0.025)*RMSE_2h,norm.ppf(0.975)*RMSE_2h]
CI_3h = [norm.ppf(0.025)*RMSE_3h,norm.ppf(0.975)*RMSE_3h]
print(f'Confidence interval 1h: {CI_1h}')
print(f'Confidence interval 2h: {CI_2h}')
print(f'Confidence interval 3h: {CI_3h}')
MAE_1h = stat.mean(np.abs(err_1h[n]) for n in range(len(err_1h)))
MAE_2h = stat.mean(np.abs(err_2h[n]) for n in range(len(err_2h)))
MAE_3h = stat.mean(np.abs(err_3h[n]) for n in range(len(err_3h)))
MAE_4h = stat.mean(np.abs(err_4h[n]) for n in range(len(err_4h)))
MAE_5h = stat.mean(np.abs(err_5h[n]) for n in range(len(err_5h)))
MAE_6h = stat.mean(np.abs(err_6h[n]) for n in range(len(err_6h)))

Confidence interval 1h: [-0.1356837401693049, 0.13568374016930487]
Confidence interval 2h: [-0.21401314081312967, 0.21401314081312964]
Confidence interval 3h: [-0.26645237089503265, 0.2664523708950326]


In [11]:
MAE_6h

0.14700931107540477

In [25]:
price=140 # euro/MWh
scale=100 #MW
penalty = 30 #euro/MWh

Rev_perf_1h=np.sum(scale*np.squeeze(p_targets1h)*price)
print("Perfect revenue {} MEuro".format(np.round(Rev_perf_1h/1e6,4)))

Rev_3m_1h=scale*np.sum(np.squeeze(p_targets1h)*price - np.abs(err_1h)*penalty)  #Euro
print("The revenue of the best model 1h ahead is {} MEuro".format(np.round(Rev_3m_1h/1e6,2)))

Rev_3m_3h=scale*np.sum(np.squeeze(p_targets3h)*price - np.abs(err_3h)*penalty)  #Euro
print("The revenue of the best model 3h ahead is {} MEuro".format(np.round(Rev_3m_3h/1e6,2)))

Rev_3m_6h=scale*np.sum(np.squeeze(p_targets6h)*price - np.abs(err_6h)*penalty) #Euro
print("The revenue of the best model 6h ahead is {} MEuro".format(np.round(Rev_3m_6h/1e6,2)))

Perfect revenue 10.2763 MEuro
The revenue of the best model 1h ahead is 9.91 MEuro
The revenue of the best model 3h ahead is 9.5 MEuro
The revenue of the best model 6h ahead is 9.15 MEuro


In [14]:
p_targets1h[:30]

[array([0.005]),
 array([0.06]),
 array([0.125]),
 array([0.18]),
 array([0.201]),
 array([0.12]),
 array([0.231]),
 array([0.406]),
 array([0.396]),
 array([0.566]),
 array([0.707]),
 array([0.807]),
 array([0.777]),
 array([0.612]),
 array([0.371]),
 array([0.135]),
 array([0.195]),
 array([0.17]),
 array([0.241]),
 array([0.201]),
 array([0.155]),
 array([0.16]),
 array([0.251]),
 array([0.326]),
 array([0.19]),
 array([0.125]),
 array([0.135]),
 array([0.11]),
 array([0.09]),
 array([0.14])]